In [ ]:
import numpy as np
import pandas as pd

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import GridSearchCV

from sklearn import metrics

# 스태킹 앙상블 모델
- 개별적인 기반 모델 + 최종 메타 모델
- 최종 메타 모델 : 개별 기반 모델에서 얻은 예측 데이터를 훈련 데이터로 사용해 학습하는 모델
- 여러 개별 모델의 예측 데이터를 각각 스태킹 형태로 결합해 최종 메타 모델의 학습용 피처 데이터 세트와 테스트용 피처 데이터 세트를 만드는 모델
- 장점 : 높은 성능 / 단점 : 어마어마어마하게 느림
- 실제 비즈니스보단 kaggle 같은 정확성만 요구되는 대회에서 많이 쓰임

## 1. 회귀
- 종속변수가 연속변수인 데이터 예측
- 스태킹 회귀 모델은 실제값과 예측값의 차이인 오류를 최소로 줄일 수 있는 선형 함수를 찾아 이를 접목하는 것
- 따라서 스태킹 회귀 모델에서 최종 메타 모델은 선형회귀 모델이어야 합니다. (Ridge / Lasso / Linear) 
- 예) 집값 예측 / 주문량 예측

In [ ]:
### 사전에 필요한 작업
# 1. 파일 불러오기
path = ''
df = pd.read_csv(path, encoding='utf-8')

# 2. 훈련용 데이터 / 테스트용 데이터 분리하기
split = 10000
target = ''

### df 확보 후 target, 즉 예측해야할 칼럼값을 맨 앞자리에 두자.
df.columns = [target, 'col1', 'col2', 'col3']
result_df = df[target]
reason_df = df.iloc[:, 1:]
X_train = reason_df[:split]
X_test = reason_df[split:]
y_train = result_df[:split]
y_test = result_df[split:]

# 3. ridge, lasso, xgbr, lgbmr 모델 만들어두기
# xgbr과 lgbmr은 best params을 찾기 위해 gridsearchCV 진행
ridge_reg = Ridge(alpha=8)
ridge_reg.fit(X_train, y_train)
lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(X_train, y_train)

params = {
    'num_leaves': [26, 27, 28, 29],
    'learning_rate': [0.07, 0.05],
    'max_depth': [-1, -2, -3],
    'n_estimators': [1000, 2000, 3000],
}

gs = GridSearchCV(LGBMRegressor(), params, scoring='neg_mean_squared_error', cv=3, n_jobs=multiprocessing.cpu_count())
gs.fit(X_train, y_train)

print("GridSearchCV 파라미터 : {}".format(gs.best_params_))
print("GridSearchCV 결과 : {}".format(gs.best_score_))

lgbm_reg = LGBMRegressor(learning_rate=gs.best_params_['learning_rate'], max_depth=gs.best_params_['max_depth'], n_estimators=gs.best_params_['n_estimators'], num_leaves=gs.best_params_['num_leaves'])
lgbm_reg.fit(X_train, y_train)


param = {
    'max_depth':[2,3,4],
    'n_estimators':range(300,600,100), #  'n_estimators':range(600,700,50) 여기에 cv 10 (이거와 별반차이가 없다.)
    'colsample_bytree':[0.5,0.7,1],
    'colsample_bylevel':[0.5,0.7,1]
}
model = XGBRegressor()
gs = GridSearchCV(estimator=model, param_grid=param, cv=3, 
                           scoring='neg_mean_squared_error',
                           n_jobs=multiprocessing.cpu_count())

gs.fit(X_train, y_train)
print("GridSearchCV 파라미터 : {}".format(gs.best_params_))
print("GridSearchCV 결과 : {}".format(gs.best_score_))


xgb_reg = XGBRegressor(n_estimators=gs.best_params_['n_estimators'], colsample_bylevel=gs.best_params_['colsample_bylevel'], learning_rate=0.1, colsample_bytree=gs.best_params_['colsample_bytree'], max_depth=gs.best_params_['max_depth'])
xgb_reg.fit(X_train, y_train)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

# 개별 기반 모델에서 최종 메타 모델이 사용할 학습 및 테스트용 데이터를 생성하기 위한 함수
def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds):
    # 지정된 n_folds값으로 KFold 생성.
    kf = KFold(n_splits=n_folds, shuffle=False, random_state=0)
    # 추후에 메타 모델이 사용할 학습 데이터 변환을 위한 넘파이 배열 초기화
    train_fold_pred = np.zeros((X_train_n.shape[0], 1))
    test_pred = np.zeros(X_test_n.shape[0], n_folds)
    print(model.__class__.__name__, 'Model 시작')

    for folder_counter, (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        # 입력된 학습 데이터에서 기반 모델이 학습/예측할 폴드 데이터 세트 추출
        print('\t 폴드세트 : ', folder_counter, '시작')
        X_tr = X_train_n[train_index]
        y_tr = y_train_n[train_index]
        X_te = X_train_n[valid_index]

        # 폴드 세트 내부에서 다시 만들어진 학습 데이터로 기반 모델의 학습 수행
        model.fit(X_tr, y_tr)
        # 폴드 세트 내부에서 다시 만들어진 검증 데이터로 기반 모델 예측 후 데이터 저장
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1, 1)
        # 입력된 원본 테스트 데이터를 폴드 세트 내 학습된 기반 모델에서 예측 후 데이터 저장
        test_pred[:, folder_counter] = model.predict(X_test_n)

    # 폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터로 생성
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1, 1)

    #train_fold_pred는 최종 메타 모델이 사용하는 학습 데이터, test_pred_mean은 테스트 데이터
    return train_fold_pred, test_pred_mean

In [ ]:
# get_stacking_base_datasets()은  리턴값은 ndarray 배열을 인자로 사용하게끔 만들었음
# DataFrame은 넘파이로 변환 후 사용
X_train_n = X_train.values
X_test_n = X_test.values
y_train_n = y_train.values

# 각 개별 기반(base) 모델이 생성한 학습용/테스트용 데이터 반환
ridge_train, ridge_test = get_stacking_base_datasets(ridge_reg, X_train_n, y_train_n, X_test_n, 5)
lasso_train, lasso_test = get_stacking_base_datasets(lasso_reg, X_train_n, y_train_n, X_test_n, 5)
xgbr_train, xgbr_test = get_stacking_base_datasets(xgb_reg, X_train_n, y_train_n, X_test_n, 5)
lgbmr_train, lgbmr_test = get_stacking_base_datasets(lgbm_reg, X_train_n, y_train_n, X_test_n, 5)

In [ ]:
# 개별 모델이 반환한 학습 및 테스트용 데이터 세트를 스태킹 형태로 결합
stack_final_X_train = np.concatenate((ridge_train, lasso_train, xgbr_train, lgbmr_train), axis=1)
stack_final_X_test = np.concatenate((ridge_test, lasso_test, xgbr_test, lgbmr_test), axis=1)

# 최종 메타 모델은 라쏘 모델 적용
meta_model_lasso = Lasso(alpha=0.0005)

# 개별 모델 예측값을 기반으로 새롭게 만들어진 학습(테스트 데이터로 메타 모델 예측 및 RMSE 측정
meta_model_lasso.fit(stack_final_X_train, y_train)
final = meta_model_lasso.predict(stack_final_X_test)
mse = metrics.mean_squared_error(y_test, final)
rmse = np.sqrt(mse)
print('스태킹 회귀 모델의 최종 RMSE 값 : {}'.format(rmse))

## 2. 분류
- 종속변수가 범주형인 데이터 예측 기법
- 성능이 비슷한 모델을 결합해 더 나은 성능 향상 도출에 활용

### 1. Basic Form
- 실제론 cv모델을 많이 사용함.
- basic form은 이해 위해 다룸!

In [ ]:
### 책에서 cancer 데이터로
### 똑같이 진행해보겠음

import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

cancer_data = load_breast_cancer

X_data = cancer_data.data
y_label = cancer_data.target

X_train, X_test, y_train, y_test = train_test_split(X_data, y_label, test_size=.2, random_state=0)

In [ ]:
# 개별 ML 모델 생성
knn_clf = KNeighborsClassifier(n_neighbors=4)
rf_clf = RandomForestClassifier(n_estimators=100, random_state=0)
dt_clf = DecisionTreeClassifier()
ada_clf = AdaBoostClassifier(n_estimators=100)
gbm_clf = GradientBoostingClassifier()
# n_estimators는 생성할 의사결정 나무 갯수를 의미한다. 이 값이 100이면 100개의 의사결정 나무를 만들고 그 중 하나

# 스태킹으로 만들어진 데이터 세트를 학습, 예측할 최종 모델
lr_final = LogisticRegression(c=10)

# 개별 모델 학습 시키기
knn_clf.fit(X_train, y_train)
rf_clf.fit(X_train, y_train)
dt_clf.fit(X_train, y_train)
ada_clf.fit(X_train, y_train)
gbm_clf.fit(X_train, y_train)

In [ ]:
# 학습된 개별 모델들이 각자 반환하는 예측 데이터 세트를 생성하고 개별 모델의 정확도 측정
knn_pred = knn_clf.predict(X_test)
rf_pred = knn_clf.predict(X_test)
dt_pred = knn_clf.predict(X_test)
ada_pred = knn_clf.predict(X_test)
gbm_pred = gbm_clf.predict(X_test)

print('KNN 정확도 : {}'.format(accuracy_score(y_test, knn_pred)))
print('랜덤포레스트 정확도 : {}'.format(accuracy_score(y_test, rf_pred)))
print('의사결정 나무 정확도 : {}'.format(accuracy_score(y_test, dt_pred)))
print('에이다부스트 정확도 : {}'.format(accuracy_score(y_test, ada_pred)))
print('그라디언트부스트 정확도 : {}'.format(accuracy_score(y_test, gbm_pred)))

In [ ]:
pred = np.array([knn_pred, rf_pred, dt_pred, ada_pred, gbm_pred])
print(pred.shape)

# transpose를 이용해 행과 열의 위치 교환. 칼럼 레벨로 각 알고리즘의 예측 결과를 피처로 만듦
pred = np.transpose(pred)
print(pred.shape)

In [ ]:
lr_final.fit(pred, y_test)
final = lr_final.predict(pred)

print('최종 메타 모델의 예측 정확도 : {}'.format(accuracy_score(y_test, final)))

### 2. KFold

In [ ]:
def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds):
    # 지정된 n_folds값으로 KFold 생성.
    kf = KFold(n_splits=n_folds, shuffle=False, random_state=0)
    # 추후에 메타 모델이 사용할 학습 데이터 변환을 위한 넘파이 배열 초기화
    train_fold_pred = np.zeros((X_train_n.shape[0], 1))
    test_pred = np.zeros(X_test_n.shape[0], n_folds)
    print(model.__class__.__name__, 'Model 시작')

    for folder_counter, (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        # 입력된 학습 데이터에서 기반 모델이 학습/예측할 폴드 데이터 세트 추출
        print('\t 폴드세트 : ', folder_counter, '시작')
        X_tr = X_train_n[train_index]
        y_tr = y_train_n[train_index]
        X_te = X_train_n[valid_index]

        # 폴드 세트 내부에서 다시 만들어진 학습 데이터로 기반 모델의 학습 수행
        model.fit(X_tr, y_tr)
        # 폴드 세트 내부에서 다시 만들어진 검증 데이터로 기반 모델 예측 후 데이터 저장
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1, 1)
        # 입력된 원본 테스트 데이터를 폴드 세트 내 학습된 기반 모델에서 예측 후 데이터 저장
        test_pred[:, folder_counter] = model.predict(X_test_n)

    # 폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터로 생성
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1, 1)

    #train_fold_pred는 최종 메타 모델이 사용하는 학습 데이터, test_pred_mean은 테스트 데이터
    return train_fold_pred, test_pred_mean

In [ ]:
knn_train, knn_test = get_stacking_base_datasets(knn_clf, X_train, y_train, X_test, 7)
rf_train, rf_test = get_stacking_base_datasets(rf_clf, X_train, y_train, X_test, 7)
dt_train, dt_test = get_stacking_base_datasets(dt_clf, X_train, y_train, X_test, 7)
ada_train, ada_test = get_stacking_base_datasets(ada_clf, X_train, y_train, X_test, 7)
gbm_train, gbm_test = get_stacking_base_datasets(gbm_clf, X_train, y_train, X_test, 7)

In [ ]:
stack_final_X_train = np.concatenate((knn_train, rf_train, dt_train, ada_train, gbm_train))
stack_final_X_test = np.concatenate((knn_test, rf_test, dt_test, ada_test, gbm_test))

In [ ]:
lr_final.fit(stack_final_X_train, y_train)
stack_final = lr_final.fit(stack_final_X_test)

print('최종 메타 모델의 예측 정확도 : {}'.format(accuracy_score(y_test, stack_final)))